Plotting the Cluster

In [1]:
import folium
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

# Load your clustered data
df = pd.read_csv("hotspot_clustered.csv")

# Create base map
m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=8)

# Generate distinct colors for each cluster
unique_clusters = sorted(df["Cluster"].unique())
num_clusters = len(unique_clusters)

colormap = cm.get_cmap("tab10", num_clusters)

cluster_colors = {
    cluster: colors.to_hex(colormap(i / (num_clusters - 1)))
    for i, cluster in enumerate(unique_clusters)
}

# Add markers by cluster
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        popup=(
            f"Cluster: {row['Cluster']}<br>"
            f"Desa: {row['Desa']}<br>"
            f"Kecamatan: {row['Kecamatan']}<br>"
            f"Kab/Kota: {row['Kab Kota']}<br>"
            f"Satelit: {row['Satelit']}<br>"
            f"Confidence: {row['Confidence']}<br>"
            f"Tanggal: {row['Tanggal']} {row['Waktu']}"
        ),
        color=cluster_colors[row["Cluster"]],
        fill=True,
        fill_color=cluster_colors[row["Cluster"]],
        fill_opacity=0.8
    ).add_to(m)

m

C:\Users\HP\AppData\Local\Temp\ipykernel_16600\1762791679.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap("tab10", num_clusters)


Mapping Road Depo, Sub Depo

In [10]:
import folium
import pandas as pd
import openrouteservice
from shapely.geometry import Point, MultiPoint
import matplotlib.cm as cm
import matplotlib.colors as colors
import time

# === 1. Inisialisasi OpenRouteService ===
client = openrouteservice.Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImUwMGE1YzgzY2Q5NTRkMTY4ODM1Y2JiNjFiMzgwMmVmIiwiaCI6Im11cm11cjY0In0=')

# === 2. Hardcode koordinat depo utama dan titik representatif per cluster ===
depo_bpbd = [-0.343085986, 111.7502586]  # Depo utama BPBD
cluster_points = {
    1: [-0.409011175, 112.1958254],
    2: [-0.592699463, 111.5859392],
    3: depo_bpbd,  # cluster 3 = depo utama
    4: [-0.569912205, 111.9740824]
}

# === 3. Baca dataset hotspot hasil klaster ===
df = pd.read_csv("hotspot_clustered.csv")

# Pastikan kolom lat/lon ada dan benar
assert "Latitude" in df.columns and "Longitude" in df.columns, "Kolom Latitude/Longitude tidak ditemukan."

# Buat kolom geometry untuk convex hull
df["geometry"] = df.apply(lambda r: Point(r["Longitude"], r["Latitude"]), axis=1)

# === 4. Setup peta dasar ===
m = folium.Map(location=depo_bpbd, zoom_start=8)

# === 5. Warna unik per cluster ===
unique_clusters = sorted(df["Cluster"].unique())
num_clusters = len(unique_clusters)
colormap = cm.get_cmap("tab10", num_clusters)
cluster_colors = {
    cluster: colors.to_hex(colormap(i / (num_clusters - 1)))
    for i, cluster in enumerate(unique_clusters)
}

# === 6. Gambarkan convex hull untuk tiap cluster ===
for cluster in unique_clusters:
    cluster_points_df = df[df["Cluster"] == cluster]
    if len(cluster_points_df) >= 3:
        try:
            hull = MultiPoint(cluster_points_df["geometry"].tolist()).convex_hull
            folium.GeoJson(
                hull.__geo_interface__,
                style_function=lambda x, color=cluster_colors[cluster]: {
                    "fillColor": color,
                    "color": color,
                    "weight": 2,
                    "fillOpacity": 0.25
                },
                tooltip=f"Cluster {cluster} Area"
            ).add_to(m)
        except Exception as e:
            print(f"Gagal membuat convex hull cluster {cluster}: {e}")

# === 7. Tambahkan titik hotspot dengan warna per cluster ===
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=4,
        color=cluster_colors[row["Cluster"]],
        fill=True,
        fill_color=cluster_colors[row["Cluster"]],
        fill_opacity=0.8,
        popup=(f"<b>Cluster:</b> {row['Cluster']}<br>"
               f"<b>Desa:</b> {row['Desa']}<br>"
               f"<b>Kecamatan:</b> {row['Kecamatan']}<br>"
               f"<b>Satelit:</b> {row['Satelit']}<br>"
               f"<b>Confidence:</b> {row['Confidence']}")
    ).add_to(m)

# === 8. Tambahkan marker cluster midpoint & depo utama ===
for cluster_id, (lat, lon) in cluster_points.items():
    if cluster_id == 3:
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color="green", icon="home"),
            popup=f"Cluster {cluster_id} - Depo Utama BPBD"
        ).add_to(m)
    else:
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color="blue", icon="info-sign"),
            popup=f"Cluster {cluster_id} - Road Midpoint"
        ).add_to(m)

# === 9. Gambarkan rute jalan nyata dari depo utama ke tiap cluster lain ===
for cluster_id, (lat, lon) in cluster_points.items():
    if cluster_id == 3:  # skip depo sendiri
        continue
    try:
        coords = [[depo_bpbd[1], depo_bpbd[0]], [lon, lat]]  # lon, lat
        route = client.directions(
            coordinates=coords,
            profile="driving-car",
            format="geojson"
        )

        # Ambil koordinat rute
        route_coords = route["features"][0]["geometry"]["coordinates"]
        route_distance = route["features"][0]["properties"]["summary"]["distance"] / 1000  # km

        # Konversi ke [lat, lon]
        route_latlons = [[c[1], c[0]] for c in route_coords]

        folium.PolyLine(
            locations=route_latlons,
            color=cluster_colors[cluster_id],
            weight=4,
            opacity=0.9,
            popup=f"BPBD → Cluster {cluster_id}<br>{route_distance:.2f} km"
        ).add_to(m)

        print(f"✅ Route BPBD → Cluster {cluster_id}: {route_distance:.2f} km")

    except Exception as e:
        print(f"❌ Gagal ambil rute ke Cluster {cluster_id}: {e}")

    time.sleep(1.5)

# === 11. Simpan hasil ke file HTML ===
m.save("bpbd_hotspot_convex_routes.html")
m


C:\Users\HP\AppData\Local\Temp\ipykernel_16600\1787704712.py:36: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap("tab10", num_clusters)


✅ Route BPBD → Cluster 1: 79.94 km
✅ Route BPBD → Cluster 2: 59.91 km
✅ Route BPBD → Cluster 4: 63.54 km
